# AI - Project 01 : Genetic
810196684

نام پروژه : رمزگشایی رمزنگاری جایگزینی


هدف پروژه : در این پروژه یک متن رمز شده به ما داده می شود که با روش جایگزینی (هر حرف الفبا را به حرف دیگری نگاشت کنیم) رمز شده است. رمزگشایی این متن را در حالت عادی با ۲۶! حالت می توان انجام داد که چون خیلی عدد بزرگی است انجام آن امکان پذیر نیست.
چون فضای حالت این مساله بسیار بزرگ است از الگوریتم ژنتیک استفاده می کنیم. بنابراین هدف از انجام این پروژه که یافتن نگشت استفاده شده برای رمز کردن متن است را به کمک این الگوریتم انجام می دهیم.



در مرحله اول کتابخانه هایی که لازم داریم را 
import
میکنیم. سپس مقادیری مانند اندازه جمعیت و احتمال کراس اور کردن و ... را تعریف کنیم.
و بعد کلاس هایی که از آن ها استفاده کردیم را تعریف میکنیم.

نام دارد که هر اینستنس از آن یک کروموزوم است( در اصل کروموزوم اصلی همان مشخصه ی دیکشنری در این اینستنس است) Key کلاس اول
هر یک از این کروموزوم ها مشخصه هایی مانند یک دیکشنری که یک نگاشت از حروف الفبا به حروف الفبا می باشد و مقدار فیتنس و لیست کلمه هایی که با این کروموزوم معنی ندارند و این که این کروموزوم جواب است یا نه را دارد. 

است. که مشخصه هایی مانند میتینگ پول و لغت نامه و جمعیت و ... را دارد Decoder کلاس بعدی



In [5]:
import re
import random
from time import time

POPULATIONNUMBER = 800
CROSSOVERPOINTSNUMBER = 6
MUTATIONPROBABILITY = 0.1
CROSSOVERPROBABILITY = 0.7
SWAPPAIRSNUMBER = 1

class Key:
    def __init__(self):
        self.key = {}
        self.hashKey = ''
        self.fitness = 0
        self.isTheAnswer = False
        self.unMeaningfulWords = []

class Decoder:
    def __init__(self, encodedText):
        self.encodedText = encodedText
        allWords = open("global_text.txt", "r")
        dictionary = set(re.findall("[a-zA-Z]+", allWords.read()))
        self.encodedWords = set(re.findall("[a-zA-Z]+", self.encodedText))
        self.dictionary = dictionary
        self.population = []
        self.hashKey = []
        self.matingPool = []
        self.answerFounded = False
        self.bestOfThisGeneration = {}
        self.maxFitnessInGeneration = 0
        self.decodedText = ''
        self.answer = None

    مشخصه های کلاس:
    self.encodedText = encodedText 
    متن رمز شده ای که از فایل ورودی می خوانیم
    allWords = open("global_text.txt", "r") 
    در تابع سازنده کلاس برای ساخت لغت نامه فایل شامل کلمات معنی دار را میخوانیم 
    dictionary = set(re.findall("[a-zA-Z]+", allWords.read()))                                                
    self.encodedWords = set(re.findall("[a-zA-Z]+", self.encodedText)) 
    در تابع سازنده فایل شامل متن رمزشده را میخوانیم
    self.dictionary = dictionary                                  
    لغت نامه
    self.population = []                                        
    جمعیت در هر نسل              
    self.hashKey = []
    مجموعه هش عضو های جمعیت اولیه برای جلوگیری از تکراری بودن کلید ها
    self.matingPool = []                                       
    میتینگ پول   
    self.answerFounded = False                
    جواب درست پیدا شده یا خیر 
    self.bestOfThisGeneration = None            
    بهترین کلید پیدا شده در نسل 
    self.decodedText = ''                      
    متن رمزگشایی شده
    self.answer = None                               
    کلید یافته شده برای رمزگشایی

مراحل کار :

پردازش داده اولیه و تولید لغت نامه : ابتدا فایل 
را می خوانیم و فقط کلمه هایی از آن که شامل حروف الفبا (بزرگ یا کوچک) هستند را در یک ست می ریزیم و  global_text.txt 
کاراکتر های دیگر را در نظر نمیگیریم و به این ترتیب لغت نامه را ایجاد می کنیم.
در واقع به جای توجه به کاراکتر های غیر کارآمد فقط به کاراکتر های کارآمد توجه می کنیم.
سپس فایل 
را می خوانیم و به همان روشی که لغت نامه را تشکیل دادیم لغت نامه ای از کلمات رمز شده می سازیمencoded_text.txt
تا برای هر کلیدی که میسازیم کلمات موجود در آن را با استفاده از آن کلید ترجمه کنیم

تعریف کروموزوم : در این پیاده سازی هر عضو از جمعیت یک اینستنس از کلاس 
می باشد که در آن کلیدی نگه داشته می شود که نگاشت از حروف الفبا به حروف الفباست  Key
این کلید همان کروموزوم در این الگوریتم است و هر حرف در آن یک یک ژن است. 
هر یک از این کروموزوم ها را برای ایجاد جمعیت اولیه به کمک تابع
انجام می دهیم که در آن یک مجموعه رندم از اعداد بین ۹۷ تا ۱۲۲ که اسکی کد های حروف الفباهستند  addKeyToPopulation 
را میسازیم (که یکتا هم باشند و تکراری نداشته باشیم)
همچنین برای هر کلیدی که ساخته می شود هش آن را میسازیم و اگر تکراری نبود و قبلا در جمعیت وجود نداشت آن را به جمعیت اولیه اضافه میکنیم

تولید جمعیت اولیه : برای تولد جمعیت اولیه در تابع  
تا وقتی که اندازه جمعیت اولیه با مقدار تعرف شده در ابتدای کد برابر نباشد کلید جدید میسازیم و به createPopulation
جمعیت اضافه می کنیم.

تولید میتینگ پول : ابتدا جمعیت اولیه را بر اساس فیتنس هایشان به صورت صعودی مرتب می کنیم. سپس با استفاده از 
به هر فرد در جمعیت یک احتمال یا رنک نسبت میدهیم و از روی این احتمالات rankSelection روش
میتینگ پول را میسازیم

تابع تناسب : در این پیاده سازی برای سنجش میزان مفید بودن یک کلید مجموع طول های کلماتی که بعد از ترجمه شدن با آن کلید در لغت نامه وجود داشتند و معنی دار بودند را محاسبه میکنیم. برای این کار از تابع 
استفاده می کنیم. در این تابع همچنین تعداد کلماتی که با کلید داده شده معنی نشده اند را  calculate fitness

می شماریم و اگر این تعداد برابر با صفر بود مشخصه مربوط به پیدا شدن جواب را

میکنیم و جواب را ست می کنیم True

همچنین برای محاسبه فیتنس برای تمام افراد جمعیت از تابع

calculateFitnessForPopulation

استفاده میکنیم و در آن ضمن محاسبه ی فیتنس برای افراد بهترین فرد موجود در جمعیت (که بیشترین فیتنس در آن جمعیت دارد) را هم ست میکنیم. همچنین اگر فردی در جمعیت باعث 
شدن مشخصه مربوط به پیدا شدن جواب بشود هم دیگر ادامه نمیدهیم و جواب درست را برمیگردانیم.

عملیات کراس اور : از روش
استفاده می کنیم و multi points
ای که به این تابع داده شده است را در تعدادی نقاط که به صورت رندم انتخاب می شوند میشکنیمparent 
سپس به صورت یکی در میان در هر فزند بخش های 
اول و دوم را قرار می دهیم parent
سپس برای این که در کلید ها ی فرزندان مپینگ تکراری نداشته باشیم و همه ی حروف الفبا را پوشش دهیم
روی هر فرزند تابع 
را صدا می زنیم. substitutionDuplicateCharInChilds

mutation :
برای انجام میوتیشن یک کلید و تعداد ژن هایی که می خواهیم جابه جا شوند را به تابع 
میدهیم و ژن را به صورت رندم انتخاب میکنیم و حرف مپ شده به آن ها را جا به جا میکنیم mutation  

که این کار (و همچنین تمام رندم انتخاب کردن ها) برای جلوگیری از سوگیری و گیر کردن در لوکال مینیمم می باشند

همچنین برای تولید نسل جدید : ابتدا میتینگ پول را شافل مبکنیم تا رندم انتخاب کنیم. سپس روی میتینگ پول دوتا دوتا جلو میرویم. دو عدد رندم بین ۰ و ۱ تولید می کنیم که یکی احتمال کراس اور کردن و دیگرا احتمال میوتیشن کردن است. سپس اگر احتمال کراس اور در بازه ای که باید می بود دو تا 
ای که از میتینگ پول انتخاب کردیم را به تابع کراس اور می دهیم. سپس  اگر احتمال میوتیشن در بازه ای کهparent
باید می بود (در همان ایف مربوط به کراس اور) تابع
را صدا می زنیم که هم کراس اور و هم میوتیشین را روی دو فرزند انجام میدهد crossoverAndMutation

در غیر این صورت هر دو

را عینا از میتینگ پول کپی میکنیمparent





In [6]:
    def addKeyToPopulation(self):
        tempKey = Key()
        rand = random.sample(range(97, 123), 26)
        hashOfKey = ""
        for i in range(len(rand)):
            tempKey.key[chr(i+97)] = chr(rand[i])
            hashOfKey += chr(i+97) + " : " + tempKey.key[chr(i+97)] + "\n"
        if hashOfKey not in self.hashKey:
            self.hashKey.append(hashOfKey)
            tempKey.hashKey = hashOfKey
        self.population.append(tempKey)

    def createPopulation(self):
        cnt = 0
        while len(self.population) != POPULATIONNUMBER:
            self.addKeyToPopulation()
            self.calculateFitness(self.population[cnt])
            cnt+=1

    def calculateFitness(self, key):
        compiledWords = []
        for word in self.encodedWords:
            tempList = list(word)
            for i in range(len(word)):
                if ord(word[i]) < 97:
                    char = chr(ord(word[i])+32)
                    smallLetter = key.key[char]
                    tempList[i] = chr(ord(smallLetter)-32)
                else:
                    tempList[i] = key.key[word[i]]
            tempStr = "".join(tempList)
            compiledWords.append(tempStr)
        lenSum = 0
        numberOfUnmeaningfull = 0
        for word in compiledWords:
            if word in self.dictionary:
                lenSum += len(word)
            else:
                key.unMeaningfulWords.append(word)
                numberOfUnmeaningfull+=1
        key.fitness = lenSum
        if numberOfUnmeaningfull == 0:
            key.isTheAnswer = True
            self.answerFounded = True
            self.answer = key
    
    def calculateFitnessForPopulation(self):
        maxFitnessInGeneration = 0
        a = Key()
        answer = None
        for individual in self.population:
            self.calculateFitness(individual)
            if individual.fitness > maxFitnessInGeneration:
                maxFitnessInGeneration = individual.fitness
                self.bestOfThisGeneration = individual
                a = individual
            if individual.isTheAnswer == True:
                self.answerFounded = True
                answer = individual
                self.answer = answer
                return answer
        return answer

    def createMatingPool(self):
        sortedByFitness = []
        sortedByFitness = sorted(self.population, key = lambda x : x.fitness, reverse = False)
        rankSum = (len(sortedByFitness)*(len(sortedByFitness)+1))/2
        checkSum = 0
        ranks = []
        for i in range(len(sortedByFitness)):
            sortedByFitness[i].rankP = round(((i+1)/rankSum)*100,3)
            ranks.append(sortedByFitness[i].rankP)
        self.matingPool = random.choices(sortedByFitness, weights = ranks, k = POPULATIONNUMBER)

    def substitutionDuplicateCharInChilds(self, childKey1,childKey2):
        alphabet = childKey1.keys()
        key1Used = set(list(childKey1.values()))
        key2Used = set(list(childKey2.values()))
        child1values = list(childKey1.values())
        child2values = list(childKey2.values())
        key1UnUsed = []
        key2UnUsed = []
        for char in alphabet:
            if char not in key1Used:
                key1UnUsed.append(char)
            if char not in key2Used:
                key2UnUsed.append(char)
        randNum1 = random.sample(range(0, len(key1UnUsed)), len(key1UnUsed))
        randNum2 = random.sample(range(0, len(key2UnUsed)), len(key2UnUsed))
        for i in range(26):
            for j in range(i+1, 26):
                if child1values[i] == child1values[j]:
                    child1values[j] = key1UnUsed[randNum1.pop(0)]
                if child2values[i] == child2values[j]:
                    child2values[j] = key2UnUsed[randNum2.pop(0)]
        childKey1 = (dict(zip(alphabet, child1values)))
        childKey2 = (dict(zip(alphabet, child2values)))
        return childKey1, childKey2

    def crossover(self, parent1, parent2):
        crossoverPoints = random.sample(range(1, 25), CROSSOVERPOINTSNUMBER)
        crossoverPoints.append(26)
        crossoverPoints.sort()
        tempKey = parent1.key
        tempKey2 = parent2.key
        parent1Values = list(tempKey.values())
        parent2Values = list(tempKey2.values())
        childKey1 = {}
        childKey2 = {}
        for i in range(26):
            childKey1[chr(i+97)] = ''
            childKey2[chr(i+97)] = ''
        prevPoint = 0
        pointIndex = 0
        for point in crossoverPoints:
            partOfParent1 = parent1Values[prevPoint:point]
            partOfParent2 = parent2Values[prevPoint:point]
            for i in range(prevPoint, point):
                if (pointIndex % 2) == 0:
                    childKey1[chr(i+97)] = partOfParent1[i-prevPoint]
                    childKey2[chr(i+97)] = partOfParent2[i-prevPoint]
                else:
                    childKey1[chr(i+97)] = partOfParent2[i-prevPoint]
                    childKey2[chr(i+97)] = partOfParent1[i-prevPoint]
            prevPoint = point
            pointIndex+=1
        childKey1, childKey2 = self.substitutionDuplicateCharInChilds(childKey1, childKey2)
        child1 = Key()
        child2 = Key()
        child1.key = childKey1
        child2.key = childKey2
        return child1, child2

    def mutation(self, child, numOfKeys):
        for i in range(numOfKeys):
            rand1 = random.randrange(97, 123)
            rand2 = random.randrange(97, 123)
            tempValue = child.key[chr(rand1)]
            child.key[chr(rand1)] = child.key[chr(rand2)]
            child.key[chr(rand1)] = tempValue
        return child

    def crossoverAndMutation(self, parent1, parent2, numOfKeys):
        child1, child2 = self.crossover(parent1, parent2)
        child1 = self.mutation(child1, numOfKeys)
        child2 = self.mutation(child2, numOfKeys)
        return child1, child2
    
    def createNextGeneration(self):
        random.shuffle(self.matingPool)
        newGeneration = []
        for i in range(0, len(self.matingPool), 2):
            crossoverProbability = random.uniform(0,1)
            mutationProbability = random.uniform(0,1)
            if crossoverProbability <= CROSSOVERPROBABILITY:
                child1 = Key()
                child2 = Key()
                if mutationProbability <= MUTATIONPROBABILITY:
                    child1, child2 = self.crossoverAndMutation(self.matingPool[i], self.matingPool[i+1], SWAPPAIRSNUMBER)
                else:
                    child1, child2 = self.crossover(self.matingPool[i], self.matingPool[i+1])
                newGeneration.append(child1)
                newGeneration.append(child2)
            else:
                newGeneration.append(self.matingPool[i])
                newGeneration.append(self.matingPool[i+1])
        return newGeneration

سپس تابع
GA
را صدا می زنیم که در آن ابتدا جمعیت اولیه را تولید میکنیم و بعد در یک حلقه که تا پیدا شدن جواب ادامه دارد
میتینگ پول و نسل جدید تولید میکنیم و فیتنس نسل جدید را محاسبه می کنیم. 

In [7]:
    def GA(self):
        self.createPopulation()
        n = 2000
        numOfGenerations = 0
        while self.answerFounded == False and n > 0:
            numOfGenerations+=1
            self.createMatingPool()
            self.population = self.createNextGeneration()
            answer = self.calculateFitnessForPopulation()
            n-=1

در آخر در تابع دیکد متن رمز شده را با کلید یافته شده رمزگشایی می کنیم

In [8]:
    def decode(self):
        self.GA()
        decodedText = ''
        for char in self.encodedText:
            if ord(char) >= 97 and ord(char) <= 122:
                decodedText += self.answer.key[char]
            elif ord(char) >= 65 and ord(char) <= 90:
                temp = chr(ord(char)+32)
                smallLetter = self.answer.key[temp]
                decodedText += smallLetter.capitalize()
            else:
                decodedText += char
        self.decodedText = decodedText
        return self.decodedText

یکی دیگر از راه هایی که برای پیاده سازی میوتیشن انجام شد :
اگر عدد تولید شده برای احتمال میوتیشن در بازه ای که باید می بود
 به اندازه یک دهم جکعیت به صورت رندم از جمعیت انتخاب میشد و میوتیشن انجام میشد
 اما در بعضی اجرا ها در لوکال مینیمم گیر می کرد 

In [ ]:
def mutationOnOneIndividual(self, child, numOfKeys):
    for i in range(numOfKeys):
        rand1 = random.randrange(97, 123)
        rand2 = random.randrange(97, 123)
        tempValue = child.key[chr(rand1)]
        child.key[chr(rand1)] = child.key[chr(rand2)]
        child.key[chr(rand1)] = tempValue
    return child
    
def mutation(self):
    for i in range(POPULATIONNUMBER/10):
        rand =  random.randrange(0, POPULATIONNUMBER)
        self.mutationOnOneIndividual(self.population[rand], SWAPPAIRSNUMBER)

تست:

In [9]:
start = time()
encoded_text = open("encoded_text.txt", "r").read()
d = Decoder(encoded_text)
decoded_text = d.decode()
print(decoded_text)
end = time()
print("Time: " + str(end - start) + "s")

AttributeError: Decoder instance has no attribute 'decode'

جواب سوالات :

اگر تعداد جمعیت افزایش یابد مدت زمانی که برای تولید هر نسل طول می کشد افزایش می یابد اما باعث می شود که زود تر به جواب نزدیک شویم. پس روی دقت و سرعت تاثیر دارد و هم چنین نباید آن را کم انتخاب کنبم چون باعث می شود که به جواب نرسیم و در لوکال مینیمم گیر کنیم و در یک کلید اشتباه بمانیم.
پس جمعیت باید اندازه اش مناسب باشد چون میتینگ پول در هر مرحله از روی آن ساخته می شود و بنابراین باید گوناگونی داشته باشد.

میوتیشن باعث می شود که از سوگیری کروموزوم ها جلوگیری شود و در لوکال مینیمم گیر نیفتیم و رندم نس را برای ما به وجود میاورد تا بتوانیم جوابمان را بهتر کنیم. پس اگر از آن استفاده نشود و فقط از کراس اور استفاده کنیم ممکن است در لوکال مینیمم گیر کنیم و رندم نس را از دست میدهیم و ممکن است به جواب نرسیم

همان طور که از تعریف کراس اور و میوتیشن پیداست کراس اور از والد های خود فرزندان بهتری می سازد و جواب بهتری تولید میکند و دقت را افزایش میدهد. اما میوتیشین رندم نس را فراهم می کند و از لوکال مینیمم جلوگیری می کند. اگر میوتیشن نباشد ممکن است الگوریتم به جواب درست نرسد اما اگر کراس اور نباشد نمیتوانیم نسل جدید را تولید کنیم و نسل جدید را از نسل قبلی به گونه ای بسازیم که غالبا بهتر از قبل باشد.

اگر رندم نس کافی نداشته باشیم ممکن است در لوکال مینیمم گیر کنیم و کروموزوم ها تغییر نکنند. که به دلیل میوتیشین کم یا میوتیشن غلط می تواند باشد.
باید تعیین احتمال و پیاده سازی میوتیشن را به درستی انجام دهیم

متن رمزگشایی شده و کلید مربوط به آن و زمان آن :

)
چون اینجا فرمتش بد شد توی فایل

output.txt

هم گذاشتم متن و زمان رو.
(

This response originally fell into a bit bucket.  I'm reposting it
just so Bill doesn't think I'm ignoring him.

In article <C4w5pv.JxD@darkside.osrhe.uoknor.edu> bil@okcforum.osrhe.edu (Bill Conner) writes:
>Jim Perry (perry@dsinc.com) wrote:
>
>[Some stuff about Biblical morality, though Bill's quote of me had little
> to do with what he goes on to say]

Bill,

I'm sorry to have been busy lately and only just be getting around to
this.

Apparently you have some fundamental confusions about atheism; I think
many of these are well addressed in the famous FAQ.  Your generalisms
are then misplaced -- atheism needn't imply materialism, or the lack
of an absolute moral system.  However, I do tend to materialism and
don't believe in absolute morality, so I'll answer your questions.

>How then can an atheist judge value? 

An atheist judges value in the same way that a theist does: according
to a personal understanding of morality.  That I don't believe in an
absolute one doesn't mean that I don't have one.  I'm just explicit,
as in the line of postings you followed up, that when I express
judgment on a moral issue I am basing my judgment on my own code
rather than claiming that it is in some absolute sense good or bad.
My moral code is not particular different from that of others around
me, be they Christians, Muslims, or atheists.  So when I say that I
object to genocide, I'm not expressing anything particularly out of
line with what my society holds.

If your were to ask why I think morality exists and has the form it
does, my answer would be mechanistic to your taste -- that a moral
code is a prerequisite for a functioning society, and that humanity
probably evolved morality as we know it as part of the evolution of
our ability to exist in large societies, thereby achieving
considerable survival advantages.  You'd probably say that God just
made the rules.  Neither of us can convince the other, but we share a
common understanding about many moral issues.  You think you get it
from your religion, I think I get it (and you get it) from early
childhood teaching.

>That you don't like what God told people to do says nothing about God
>or God's commands, it says only that there was an electrical event in your
>nervous system that created an emotional state that your mind coupled
>with a pre-existing thought-set to form that reaction. 

I think you've been reading the wrong sort of comic books, but in
prying through the gobbledygook I basically agree with what you're
saying.  I do believe that my mental reactions to stimuli such as "God
commanded the genocide of the Canaanites" is mechanistic, but of
course I think that's true of you as well.  My reaction has little to
do with whether God exists or even with whether I think he does, but
if a god existed who commanded genocide, I could not consider him
good, which is supposedly an attribute of God.

>All of this being so, you have excluded
>yourself from any discussion of values, right, wrong, goood, evil,
>etc. and cannot participate. Your opinion about the Bible can have no
>weight whatsoever.

Hmm.  Yes, I think some heavy FAQ-reading would do you some good.  I
have as much place discussing values etc. as any other person.  In
fact, I can actually accomplish something in such a discussion, by
framing the questions in terms of reason: for instance, it is clear
that in an environment where neighboring tribes periodically attempt
to wipe each other out based on imagined divine commands, then the
quality of life will be generally poor, so a system that fosters
coexistence is superior, if quality of life is an agreed goal.  An
absolutist, on the other hand, can only thump those portions of a
Bible they happen to agree with, and say "this is good", even if the
act in question is unequivocally bad by the standards of everyone in
the discussion.  The attempt to define someone or a group of people as
"excluded from discussion", such that they "cannot participate", and
their opinions given "no weight whatsoever" is the lowest form or
reasoning (ad hominem/poisoning the well), and presumably the resort
of someone who can't rationally defend their own ideas of right,
wrong, and the Bible.
-- 
Jim Perry   perry@dsinc.com   Decision Support, Inc., Matthews NC
These are my opinions.  For a nominal fee, they can be yours.

-- 
Jim Perry   perry@dsinc.com   Decision Support, Inc., Matthews NC
These are my opinions.  For a nominal fee, they can be yours.

Time: 23.789209127426147s

{'a': 'o', 'b': 'r', 'c': 's', 'd': 'f', 'e': 'w', 'f': 'm', 'g': 'b', 'h': 't', 'i': 'i', 'j': 'z', 'k': 'g', 'l': 'h', 'm': 'k', 'n': 'n', 'o': 'v', 'p': 'e', 'q': 'l', 'r': 'p', 's': 'd', 't': 'j', 'u': 'c', 'v': 'u', 'w': 'y', 'x': 'q', 'y': 'a', 'z': 'x'}